In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [2]:
df_csv = pd.read_csv('df_past_order_with_cluster_im.csv', index_col=0, parse_dates=True)
df_csv.drop('SKU', axis=1, inplace=True)
#转置
df_csv = df_csv.T

/tmp/ipykernel_15664/2668064132.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_csv = pd.read_csv('df_past_order_with_cluster_im.csv', index_col=0, parse_dates=True)


In [3]:
#将数据集划分为训练集和测试集
train = df_csv.loc['2016-06-01':'2016-06-18']   
test = df_csv.loc['2016-06-18':'2016-06-23']


In [11]:
for col in train:
    #从第二列开始 提取每一列数据进行预测
    train_data = train[col]
    test_data = test[col]
    train_data_array = train_data.values.flatten()
    train_data_array = train_data_array.astype('float32')
    test_data_array = test_data.values.flatten()
    test_data_array = test_data_array.astype('float32') 
    #检验数据是否平稳
    adf_result = adfuller(train_data_array)
    print(f'ADF Statistic: {adf_result[0]}')
    print(f'p-value: {adf_result[1]}')
    print(f'Critical Values: {adf_result[4]}')
    #如果数据不平稳，进行差分处理
    if adf_result[1] > 0.05:
        train_data_diff = np.diff(train_data_array)
    else:
        train_data_diff = train_data_array
    #建立ARIMA模型
    model = ARIMA(train_data_diff, order=(6, 4, 1))
    model_fit = model.fit()
    #预测
    pred = model_fit.forecast(steps=5)
    #计算RMSE
    if adf_result[1] > 0.05:
        pred = np.r_[train_data_array[-1], pred].cumsum()[1:]
    else:
        pred = pred
    test_data_array = test_data_array[:len(pred)]
    # 计算RMSE
    rmse = np.sqrt(mean_squared_error(test_data_array, pred))

    print(f'预测值：{pred}')
    print(f'真实值：{test_data.values}')
    print(f'RMSE: {rmse}')
    print('-------------------------')

ADF Statistic: -3.8446077249963695
p-value: 0.002482512301067748
Critical Values: {'1%': -3.9644434814814815, '5%': -3.0849081481481484, '10%': -2.6818144444444445}


/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary startin

预测值：[346.53283193 251.81392608  78.84413699  51.24813764 -27.66904588]
真实值：[484.  559.  499.9 457.5 448.8 449. ]
RMSE: 369.4893385912745
-------------------------
ADF Statistic: -0.9966860880472832
p-value: 0.7544710545893367
Critical Values: {'1%': -4.331573, '5%': -3.23295, '10%': -2.7487}
预测值：[15.72976772 11.0414618   2.86580473  2.37837479  3.62182573]
真实值：[12.3 17.8 18.4  9.5  8.1  8.5]
RMSE: 8.596789379777913
-------------------------
ADF Statistic: -2.4392998070525187
p-value: 0.13095007616156307
Critical Values: {'1%': -4.331573, '5%': -3.23295, '10%': -2.7487}


/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoreg

预测值：[  48.07727922   27.048053    -33.22080242  -78.84054968 -110.36606621]
真实值：[55.1 71.9 70.9 43.6 32.5 44.6]
RMSE: 98.28993680930984
-------------------------
ADF Statistic: -15.614677502942296
p-value: 1.762106618716181e-28
Critical Values: {'1%': -4.137829282407408, '5%': -3.1549724074074077, '10%': -2.7144769444444443}
预测值：[28.86651604 30.94338778 17.82152897 20.04945409 23.92544683]
真实值：[19.  28.5 28.2 11.3  9.8 10.3]
RMSE: 9.870280092697449
-------------------------
ADF Statistic: -7.253070964060941
p-value: 1.7593620262495913e-10
Critical Values: {'1%': -4.137829282407408, '5%': -3.1549724074074077, '10%': -2.7144769444444443}


...

/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoreg

预测值：[ 49.87138471  31.66413624   3.6562194  -13.24563907 -16.40195852]
真实值：[55.2 67.9 72.4 48.8 46.4 37.8]
RMSE: 52.65147116905842
-------------------------
ADF Statistic: -5.443275219406856
p-value: 2.7458934932492113e-06
Critical Values: {'1%': -4.137829282407408, '5%': -3.1549724074074077, '10%': -2.7144769444444443}
预测值：[113.51625149 141.36284641 121.42428726 127.39106801 157.22981347]
真实值：[68.8 91.3 86.9 48.3 51.2 52. ]
RMSE: 68.11090339697869
-------------------------
ADF Statistic: 0.1414137771679542
p-value: 0.9687774778123668
Critical Values: {'1%': -4.331573, '5%': -3.23295, '10%': -2.7487}
预测值：[ 26.58382819  49.92328149  80.99486071 127.7390388  195.77075698]
真实值：[12.4 16.3 16.4  9.4  8.2  9.3]
RMSE: 104.58596399334927
-------------------------
ADF Statistic: -6.169392071204441
p-value: 6.863822699178636e-08
Critical Values: {'1%': -4.331573, '5%': -3.23295, '10%': -2.7487}
预测值：[143.94175511 137.27180511 139.84841417 157.48024301 167.97555454]
真实值：[ 99.6 122.2 136.5  79.7  6

/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary startin

预测值：[34.51165533 42.82411191 47.11408838 62.117974   87.18451317]
真实值：[21.9 30.5 26.7 15.7 15.3 13. ]
RMSE: 40.12401799765714
-------------------------


/home/thea/anaconda3/envs/hh_t/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
